### Example for NaCl cell optimisation with CG as implemented in Conquest

In [ ]:
import os
import shutil
import subprocess
import matplotlib.pyplot as plt
import pickle

from distutils.spawn import find_executable
from numpy import amax, amin, size, array, trapz, floor

from ase.units import Hartree
from ase.build import bulk
from ase.calculators.conquest import Conquest
from ase.io.conquest import Conquest_orthorhombic_check
from ase.io.conquest import get_fermi_level
from ase.io.conquest_f_rework_output_ase import read_conquest

from ase.dft.bandgap import bandgap
from ase.dft.dos import DOS
from ase.visualize import view
from ase.spacegroup import get_spacegroup
from ase import Atoms

from cq_ase_external_lib import print_struct_data
from cq_ase_external_lib import get_gapwind

In [ ]:
%%bash 
ase --version

#### Check if visualisation tools are installed

In [ ]:
# Add exe name 
cmd_vis = {'xcrysden' : False}

for cmd, state in cmd_vis.items():
    if ( not shutil.which(cmd) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   
        cmd_vis[cmd] = True
        
# For MacOSX add the path of the name of the app
cmd_app = {'VESTA'      : '/Applications/VESTA/VESTA.app/Contents/MacOS/VESTA',
           'VMDLauncher': '/Applications/VMD_1.9.2.app/Contents/MacOS/VMDLauncher'}

for cmd, path in cmd_app.items():
    if ( not find_executable(path) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   

#### Directory for storing calculation files

In [ ]:
working_directory = 'cq_example_rocksalt_optcell_cq'

# Test if `working_directory` exists ? If not create it
if ( not os.path.isdir(working_directory) ):
    os.makedirs(working_directory)

#### Define Conquest environment

In [ ]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 /Users/lioneltruflandier/CONQUEST-release-f-rework-output-ase/src/Conquest'
os.environ['CQ_PP_PATH'] = '/Users/lioneltruflandier/Conquest-develop-outdated/pseudo-and-pao'
os.environ['CQ_GEN_BASIS_CMD'] = '/Users/lioneltruflandier/CONQUEST-release-develop/tools/BasisGeneration/MakeIonFiles'

#### Setup Conquest basis set

In [ ]:
basis = {'Na' : {'gen_basis'            : True,
                 'basis_size'           : 'medium',
                 'pseudopotential_type' : 'hamann'},
         'Cl' : {'gen_basis'            : False,
                 'basis_size'           : 'medium',
                 'pseudopotential_type' : 'hamann'}
        }

val_elec = {'Na': 9, 'Cl': 7}

#### Generate rocksalt cell from CIF file data

- Choose either primitive or conventional cell

In [ ]:
primitive = True

- Generate rocksalt struture with $a=5.71$ Ang.

In [ ]:
if ( primitive ):
    # For primitive cell
    struct = bulk('NaCl', crystalstructure='rocksalt', a=5.71, cubic=False)
    # Conquest can only handle orthorhombic cells ; check
    struct = Conquest_orthorhombic_check(struct,verbose=False)
    # warning must be printed and 'struct' is modified to conventional cell
    
else:
    # For conventional cell
    struct = bulk('NaCl', crystalstructure='rocksalt', a=5.71, cubic=True)
    # Conquest can only handle orthorhombic cells ; check
    struct = Conquest_orthorhombic_check(struct,verbose=False)
    # no warning...

#### Compute the number of states
- Given `struct`and the `val_elec`compute the total number of electrons

In [ ]:
# Get atom labels
atom_names = struct.get_chemical_symbols()

# Compute total valence electrons
tot_val = 0
for i in range(len(struct)):
    for key, val in val_elec.items():
        if ( key == atom_names[i] ):
            tot_val = tot_val + val

print('Number of valence electron in the unit cell = ', tot_val)

- Compute number(s) of occupied states

In [ ]:
n_occ = []
if ( tot_val % 2 == 0):
    # paired electrons
    n_occ.append(int(tot_val/2))
else:
    # unpaired electrons
    n_occ.append(int((tot_val-1)/2))
    n_occ.append(int((tot_val+1)/2))
    
print()
print('Number of occupied bands = ', n_occ)

#### Extract and print main structural data from Atoms object (here `struct`)

In [ ]:
print_struct_data(struct,verbose=1)

- Space group ; Brillouin zone of space group [225](https://www.cryst.ehu.es/cgi-bin/cryst/programs/nph-kv-list?gnum=225&fig=fm3qmf&what=data)

In [ ]:
sg = get_spacegroup(struct)
print('Space group: {} ({}) '.format(sg.symbol,sg.no))

- Save input structure as VASP `POSCAR` file & `CIF` file (for checking purpose)

In [ ]:
struct.write(working_directory+'/input.vasp')
struct.write(working_directory+'/input.cif')

- run `VESTA` to check input (if possible) from `input.cif`

In [ ]:
#if ( 'VESTA' in cmd_app ):
#    subprocess.run([cmd_app['VESTA'], working_directory+'/'+'input.cif'])

#### Setup calculation using Conquest as calculator

In [ ]:
cutoff  =  80.0
kpoints = [3,3,3]
fxc     = 'PBE'

conquest_flags = {'IO.WriteOutToASEFile': True}

conquest_flags.update({'AtomMove.TypeOfRun'   : 'cg',    # optimization algorithm
                       'AtomMove.OptCell'     : True,
                       'AtomMove.OptCellMethod'    : 1,
                       'AtomMove.TargetPressure'   : 1e-5,
                       'AtomMove.EnthalpyTolerance': 1e-6,
                       'AtomMove.StressTolerance'  : 0.001,
                      })    

calc = Conquest(directory      = working_directory,
                grid_cutoff    = cutoff,
                self_consistent= True,
                xc    = fxc,
                basis = basis,
                kpts  = kpoints,
                nspin = 1,
                **conquest_flags)

struct.calc = calc

#### Launch structure optimization

In [ ]:
cq_dft_energy = struct.get_potential_energy()

#### Post-process `trajectory.xsf` file generated by Conquest

In [ ]:
traj_file='trajectory.xsf'
traj_file=working_directory+'/'+traj_file

if ( os.path.isfile(traj_file) ):
    # cq_postproc_xsf output last structure from xsf file
    struct_last = cq_postproc_xsf(traj_file,len(struct))

    struct_last.write(working_directory+'/output.cif')
    struct_last.write(working_directory+'/output.xsf')
else:
    print('WARNING: {} not found !'.format(traj_file))

- run `xcrysden` to visualize the optimization movie (if possible)

In [ ]:
#if ( cmd_vis['xcrysden'] and os.path.isfile(traj_file) ):
#    subprocess.run(['xcrysden', '--axsf', traj_file])

- or read last structure from Conquest  `coord_next.dat` 

In [ ]:
last_struct='coord_next.dat'
last_struct=working_directory+'/'+last_struct

if( os.path.isfile(last_struct) ):

    struct_last = read_conquest(last_struct, fractional=True, atomic_order=['Na','Cl'])
    
    struct_last.write(working_directory+'/output_.cif')
    struct_last.write(working_directory+'/output_.xsf')

#### Compute final lattice constant

In [ ]:
exp_lattice = 5.64
thr_lattice = (8*struct_last.get_volume()/len(struct_last))**(1.0/3.0)
print('exp lattice contant = {:8.4f}'.format(exp_lattice))
print('th  lattice contant = {:8.4f}'.format(thr_lattice))